In [8]:
import pandas as pd
import pyarrow.parquet as pq

In [2]:
ANON_CONTROL_MUSICS = 'data/anon_control_musics.csv'
ANON_CONTROL_TWEETS = 'data/anon_control_tweets.csv'
ANON_DISORDER_MUSICS = 'data/anon_disorder_musics.csv'
ANON_DISORDER_TWEETS = 'data/anon_disorder_tweets.csv'

In [3]:
control_music = pd.read_csv(ANON_CONTROL_MUSICS, dtype=str)
control_tweets = pd.read_csv(ANON_CONTROL_TWEETS, dtype=str)
disorder_music = pd.read_csv(ANON_DISORDER_MUSICS, dtype=str)
disorder_tweets = pd.read_csv(ANON_DISORDER_TWEETS, dtype=str)

In [9]:
control_music.to_parquet('data/anon_control_musics.parquet')
control_tweets.to_parquet('data/anon_control_tweets.parquet')
disorder_music.to_parquet('data/anon_disorder_musics.parquet')
disorder_tweets.to_parquet('data/anon_disorder_tweets.parquet')